# 使用Spark和Flask的在线电影推荐服务-构建Web服务

本教程详细介绍了如何在Web服务中使用Spark机器学习模型，甚至其他类型的数据分析对象。 这为在线预测、推荐等提供了方便。借助Python语言，且由于Spark自身的Python功能以及基于Python的框架（例如Flask），我们使此任务非常容易。

本教程可以独立使用，以在任何种类的Spark模型之上构建Web服务。 但是，它与我们使用的基于MovieLens数据集构建电影推荐模型的Spark MLlib的教程强有力地结合在一起。 这样你将能够开发出完整的在线电影推荐服务。

我们完整的Web服务包含三个Python文件：
- engine.py 定义推荐引擎，包装所有与Spark相关的计算。
- app.py 是Flask web应用程序，它在引擎周围定义了类似RESTful的API。
- server.py 使用前一个创建Spark context和Flask web app后，初始化CherryPy Web服务器。
但是，让我们详细解释它们中的每一个，以及使用Spark作为计算引擎部署这样的系统的特殊性。我们将重点介绍如何在我们处理的web context中使用Spark模型。有关MovieLens数据以及如何使用Spark构建模型的说明，请阅读《基于Spark和Flask构建的在线电影推荐系统》教程。

## 一个推荐引擎

电影推荐Web服务的核心是推荐引擎（例如：我们最终部署在engine.py）。它由RecommendationEngine类表示，本节将逐步描述其功能和实现方式。

### 开始这个引擎

引擎初始化后，我们需要首次了解ALS模型。 可选地（在这里我们不会做），我们可能正在加载先前持久的模型，以便将其用于推荐。 此外，我们可能需要加载或预先计算任何RDD，这些RDD将在以后用于做出推荐。

我们将在RecommendationEngine类的__init__方法中进行此类操作（使用两个私有方法）。 在这种情况下，我们将不会节省任何时间。 每当引擎创建时，我们都会重复整个过程。

In [2]:
import os
from pyspark.mllib.recommendation import ALS
 
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)



class RecommendationEngine:
    """A movie recommendation engine
    """
 
    def __count_and_average_ratings(self):
        """Updates the movies ratings counts from 
        the current data self.ratings_RDD
        """
        logger.info("Counting movie ratings...")
        movie_ID_with_ratings_RDD = self.ratings_RDD.map(lambda x: (x[1], x[2])).groupByKey()
        movie_ID_with_avg_ratings_RDD = movie_ID_with_ratings_RDD.map(get_counts_and_averages)
        self.movies_rating_counts_RDD = movie_ID_with_avg_ratings_RDD.map(lambda x: (x[0], x[1][0]))
 
 
    def __train_model(self):
        """Train the ALS model with the current dataset
        """
        logger.info("Training the ALS model...")
        self.model = ALS.train(self.ratings_RDD, self.rank, seed=self.seed,
                               iterations=self.iterations, lambda_=self.regularization_parameter)
        logger.info("ALS model built!")
 
 
    def __init__(self, sc, dataset_path):
        """Init the recommendation engine given a Spark context and a dataset path
        """
 
        logger.info("Starting up the Recommendation Engine: ")
 
        self.sc = sc
 
        # Load ratings data for later use
        logger.info("Loading Ratings data...")
        ratings_file_path = os.path.join(dataset_path, 'ratings.csv')
        ratings_raw_RDD = self.sc.textFile(ratings_file_path)
        ratings_raw_data_header = ratings_raw_RDD.take(1)[0]
        self.ratings_RDD = ratings_raw_RDD.filter(lambda line: line!=ratings_raw_data_header)\
            .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2]))).cache()
        # Load movies data for later use
        logger.info("Loading Movies data...")
        movies_file_path = os.path.join(dataset_path, 'movies.csv')
        movies_raw_RDD = self.sc.textFile(movies_file_path)
        movies_raw_data_header = movies_raw_RDD.take(1)[0]
        self.movies_RDD = movies_raw_RDD.filter(lambda line: line!=movies_raw_data_header)\
            .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1],tokens[2])).cache()
        self.movies_titles_RDD = self.movies_RDD.map(lambda x: (int(x[0]),x[1])).cache()
        # Pre-calculate movies ratings counts
        self.__count_and_average_ratings()
 
        # Train the model
        self.rank = 8
        self.seed = 5
        self.iterations = 10
        self.regularization_parameter = 0.1
        self.__train_model()

关于__init__和两个私有方法的所有代码已在《基于Spark和Flask构建的在线电影推荐系统》构建模型的教程中进行了说明。

### 添加一个新的评分

使用协同过滤和Spark的交替最小二乘时，我们需要为每一批新的用户评分重新计算预测模型。我们在之前的有关构建模型的教程中对此进行了解释。

In [ ]:
def add_ratings(self, ratings):
    """Add additional movie ratings in the format (user_id, movie_id, rating)
    """
    # Convert ratings to an RDD
    new_ratings_RDD = self.sc.parallelize(ratings)
    # Add new ratings to the existing ones
    self.ratings_RDD = self.ratings_RDD.union(new_ratings_RDD)
    # Re-compute movie ratings count
    self.__count_and_average_ratings()
    # Re-train the ALS model with the new ratings
    self.__train_model()

    return ratings

# Attach the function to a class method
RecommendationEngine.add_ratings = add_ratings

### 开始推荐

我们还在有关构建影片推荐系统的教程中解释了如何使用ALS模型进行推荐。在这里，我们将基本上重复等效的代码，将它们包装在RecommendationEnginer类的方法中，并把用于每种预测方法定义为私有方法。

In [ ]:
def __predict_ratings(self, user_and_movie_RDD):
    """Gets predictions for a given (userID, movieID) formatted RDD
    Returns: an RDD with format (movieTitle, movieRating, numRatings)
    """
    predicted_RDD = self.model.predictAll(user_and_movie_RDD)
    predicted_rating_RDD = predicted_RDD.map(lambda x: (x.product, x.rating))
    predicted_rating_title_and_count_RDD = \
        predicted_rating_RDD.join(self.movies_titles_RDD).join(self.movies_rating_counts_RDD)
    predicted_rating_title_and_count_RDD = \
        predicted_rating_title_and_count_RDD.map(lambda r: (r[1][0][1], r[1][0][0], r[1][1]))

    return predicted_rating_title_and_count_RDD
    
def get_top_ratings(self, user_id, movies_count):
    """Recommends up to movies_count top unrated movies to user_id
    """
    # Get pairs of (userID, movieID) for user_id unrated movies
    user_unrated_movies_RDD = self.ratings_RDD.filter(lambda rating: not rating[1]==user_id).map(lambda x: (user_id, x[1]))
    # Get predicted ratings
    ratings = self.__predict_ratings(user_unrated_movies_RDD).filter(lambda r: r[2]>=25).takeOrdered(movies_count, key=lambda x: -x[1])

    return ratings

# Attach the functions to class methods
RecommendationEngine.__predict_ratings = __predict_ratings
RecommendationEngine.get_top_ratings = get_top_ratings

除了获得靠前未评分电影外，我们还将希望获得指定电影的评分。我们将使用RecommendationEngine类中的新方法来实现。

In [ ]:
def get_ratings_for_movie_ids(self, user_id, movie_ids):
    """Given a user_id and a list of movie_ids, predict ratings for them 
    """
    requested_movies_RDD = self.sc.parallelize(movie_ids).map(lambda x: (user_id, x))
    # Get predicted ratings
    ratings = self.__predict_ratings(requested_movies_RDD).collect()

    return ratings

# Attach the function to a class method
RecommendationEngine.get_ratings_for_movie_ids = get_ratings_for_movie_ids

## 使用Flask围绕我们的引擎构建Web API

Flask是Python的网络微框架。只需导入脚本并使用一些注释将我们的服务端点与Python函数相关联，就可以很容易地启动Web API。在我们的案例中，我们将在一些端点周围包装我们的RecommendationEngine方法，并与Web客户端交换JSON格式的数据。

实际上很简单，我们将在这里显示整个app.py，而不是一步一步地显示。

In [ ]:
from flask import Blueprint
main = Blueprint('main', __name__)
 
import json
from engine import RecommendationEngine
 
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
 
from flask import Flask, request
 
@main.route("/<int:user_id>/ratings/top/<int:count>", methods=["GET"])
def top_ratings(user_id, count):
    logger.debug("User %s TOP ratings requested", user_id)
    top_ratings = recommendation_engine.get_top_ratings(user_id,count)
    return json.dumps(top_ratings)
 
@main.route("/<int:user_id>/ratings/<int:movie_id>", methods=["GET"])
def movie_ratings(user_id, movie_id):
    logger.debug("User %s rating requested for movie %s", user_id, movie_id)
    ratings = recommendation_engine.get_ratings_for_movie_ids(user_id, [movie_id])
    return json.dumps(ratings)
 
 
@main.route("/<int:user_id>/ratings", methods = ["POST"])
def add_ratings(user_id):
    # get the ratings from the Flask POST request object
    ratings_list = request.form.keys()[0].strip().split("\n")
    ratings_list = map(lambda x: x.split(","), ratings_list)
    # create a list with the format required by the negine (user_id, movie_id, rating)
    ratings = map(lambda x: (user_id, int(x[0]), float(x[1])), ratings_list)
    # add them to the model using then engine API
    recommendation_engine.add_ratings(ratings)
 
    return json.dumps(ratings)
 
 
def create_app(spark_context, dataset_path):
    global recommendation_engine 
 
    recommendation_engine = RecommendationEngine(spark_context, dataset_path)    
    
    app = Flask(__name__)
    app.register_blueprint(main)
    return app

基本上，我们使用该应用程序如下：

- 当我们调用create_app时，做一些初始化的事情。在这里创建了RecommendationEngine对象，然后我们将上面定义的@main.route注释关联起来。每个注释由(Flask docs)定义:
     + 路由，即它的URL，并且可能包含<>之间的参数。它们被映射到函数参数。
     + HTTP可用方法的列表。

- 定义了三种注释，它们与三种RecommendationEngine方法相对应:
    + GET /<user_id>/ratings/top 从引擎获得最佳推荐。
    + GET /<user_id>/ratings 获得单个电影的预测评分。
    + POST /<user_id>/ratings 添加新的评分。 格式是一系列行（以换行符结尾），每行由movie_id和ratingg构成且由逗号分隔。 例如，以下文件对应
    于在构建模型的教程中用作示例的十个新用户评分：
    260,9  
    1,8  
    16,7  
    25,8  
    32,9  
    335,4  
    379,3  
    296,7  
    858,10  
    50,8

## 使用CherryPy部署WSGI服务器

除其他外，CherryPy框架还具有可靠的，兼容HTTP/1.1的WSGI线程池化Web服务器。一次运行多个HTTP服务器（例如在多个端口上）也很容易。所有这些使其成为轻松为我们的在线推荐服务部署生产Web服务器的理想人选。

我们对CherryPy服务器的使用相对简单。其次，我们将在此处显示完整的server.py脚本，然后对其进行一些解释。

In [ ]:
import time, sys, cherrypy, os
from paste.translogger import TransLogger
from app import create_app
from pyspark import SparkContext, SparkConf
 
def init_spark_context():
    # load spark context
    conf = SparkConf().setAppName("movie_recommendation-server")
    # IMPORTANT: pass aditional Python modules to each worker
    sc = SparkContext(conf=conf, pyFiles=['engine.py', 'app.py'])
 
    return sc
 
 
def run_server(app):
 
    # Enable WSGI access logging via Paste
    app_logged = TransLogger(app)
 
    # Mount the WSGI callable object (app) on the root directory
    cherrypy.tree.graft(app_logged, '/')
 
    # Set the configuration of the web server
    cherrypy.config.update({
        'engine.autoreload.on': True,
        'log.screen': True,
        'server.socket_port': 5432,
        'server.socket_host': '0.0.0.0'
    })
 
    # Start the CherryPy WSGI web server
    cherrypy.engine.start()
    cherrypy.engine.block()
 
 
if __name__ == "__main__":
    # Init spark context and load libraries
    sc = init_spark_context()
    dataset_path = os.path.join('datasets', 'ml-latest')
    app = create_app(sc, dataset_path)
 
    # start web server
    run_server(app)

这是CherryPy的标准用法。如果我们看一下__main__入口点，我们会做三件事：
- 根据函数init_spark_context中的定义创建一个spark context，并在其中传递其它Python模块。
- 创建Flask app，调用我们在app.py中定义的create_app。 
- 运行服务器本身。
请参阅以下有关启动服务器的部分。

## 使用Spark运行服务

为了使服务器能够在访问Spark context和集群的同时运行，我们需要使用spark-submit将server.py文件提交到pySpark。Spark docummentaion中更好地解释了使用此命令时的不同参数。在我们的案例中，我们将使用以下内容。
~/spark-1.3.1-bin-hadoop2.6/bin/spark-submit --master spark://169.254.206.2:7077 --total-executor-cores 14 --executor-memory 6g server.py
重要的位是：
- 用spark-submit，没有直接用pyspark。
- --master参数必须指向您的Spark集群设置（可以是本地的）。
- 您可以传递其他配置参数，例如--total-executor-cores和--executor-memory

您将看到类似以下的输出：
INFO:engine:Starting up the Recommendation Engine: INFO:engine:Loading Ratings data... INFO:engine:Loading Movies data... INFO:engine:Counting movie ratings... INFO:engine:Training the ALS model... ... More Spark and CherryPy logging INFO:engine:ALS model built! [05/Jul/2015:14:06:29] ENGINE Bus STARTING [05/Jul/2015:14:06:29] ENGINE Started monitor thread 'Autoreloader'. [05/Jul/2015:14:06:29] ENGINE Started monitor thread '_TimeoutMonitor'. [05/Jul/2015:14:06:29] ENGINE Serving on http://0.0.0.0:5432 [05/Jul/2015:14:06:29] ENGINE Bus STARTED

### 使用多个脚本和Spark提交时的一些注意事项

在像这样的部署中使用Spark时，我们需要解决两个问题。第一个是Spark集群是从启动Python脚本的Spark Master编排的Workers的分布式环境。这意味着Master是唯一可以访问提交的脚本和本地其他文件的Master。如果我们希望工作人员能够访问其他导入的Python模块，则它们必须是您的Python发行版的一部分，或者我们需要隐式传递它们。我们在创建SparkContext对象时通过使用pyFiles = ['engine.py'，'app.py']参数来执行此操作。

第二个问题与上一个问题有关，但比较棘手。在Spark中，当使用转换（例如在RDD上映射）时，我们无法引用在可执行上下文中全局不可用的其他RDD或对象。例如，我们不能引用类实例变量。因此，我们定义了在RecommendationEgine类之外传递给RDD转换的所有函数。

### 使用这个服务

现在，使用与构建模型教程相同的数据来尝试该服务。也就是说，首先我们要添加评分，然后我们将获得最高评分和个人评分。

### 发布新评分

因此首先，我们需要按照上一节中的说明运行服务。一旦运行，我们将使用curl从外壳发布新的评分。如果当前文件夹中有文件user_ratings.file（请参阅下面的获取源代码），只需执行以下命令。
curl --data-binary @user_ratings.file http://